In [40]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import sqlite3
import re
import pdfplumber
import pdfplumber
from flask import Flask, request, jsonify
import base64

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly", "https://www.googleapis.com/auth/gmail.compose", "https://www.googleapis.com/auth/gmail.send", "https://www.googleapis.com/auth/gmail.modify"]

def get_unread_emails_with_subject(service, user_id, subject):
    try:
        query = f'is:unread subject:{subject}'
        results = service.users().messages().list(userId=user_id, q=query).execute()
        messages = results.get('messages', [])
        return messages
    except HttpError as error:
        print(f"An error occurred: {error}")
        return []

def get_attachments(service, user_id, msg_id, store_dir):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        for part in message['payload']['parts']:
            if part['filename']:
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body']['attachmentId']
                    att = service.users().messages().attachments().get(userId=user_id, messageId=msg_id, id=att_id).execute()
                    data = att['data']
                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                
                filename = part['filename']
                path = os.path.join(store_dir, filename)
                if os.path.exists(path):
                    base, extension = os.path.splitext(filename)
                    counter = 1
                    while os.path.exists(path):
                        filename = f"{base}_{counter}{extension}"
                        path = os.path.join(store_dir, filename)
                        counter += 1
                
                with open(path, 'wb') as f:
                    f.write(file_data)
                print(f'Attachment {filename} saved to {path}')
    except HttpError as error:
        print(f"An error occurred: {error}")

def mark_as_read(service, user_id, msg_id):
    try:
        service.users().messages().modify(userId=user_id, id=msg_id, body={'removeLabelIds': ['UNREAD']}).execute()
        print(f'Message {msg_id} marked as read.')
    except HttpError as error:
        print(f"An error occurred: {error}")


def get_db_connection():
    conn = sqlite3.connect('job_requirements.db')
    conn.row_factory = sqlite3.Row
    return conn


def calculate_score(job, experience, qualifications, education):
    conn = get_db_connection()
    job_req = conn.execute('SELECT * FROM job_requirements WHERE jobname = ?', (job,)).fetchone()
    conn.close()

    if job_req is None:
        return 0, f"Initiativ + {job}"

    weight_experience = job_req['weight_experience']
    weight_qualifications = job_req['weight_qualifications']
    weight_education = job_req['weight_education']

    # Normalize weights to sum to 1
    total_weight = weight_experience + weight_qualifications + weight_education
    weight_experience /= total_weight
    weight_qualifications /= total_weight
    weight_education /= total_weight

    # Berechnung des Scores
    score = 0

    # Berufserfahrung bewerten
    req_experience = job_req['experience']
    job_experience = 0

    # Regex für die spezifische Joberfahrung
    job_experience_matches = re.findall(rf'(\d+)\s*(Jahre|Monate)?\s*(als|in|Tätigkeit)?\s*{job}', experience, re.IGNORECASE)
    for match in job_experience_matches:
        years_or_months = int(match[0])
        if 'Monate' in match[1]:
            years_or_months /= 12  # Monate in Jahre umrechnen
        job_experience += years_or_months

    # Berechnung des Scores basierend auf dem Verhältnis der Erfahrung
    min_years = int(re.findall(r'\d+', req_experience)[0])
    if job_experience >= min_years:
        # Volle Punkte und zusätzliche Punkte für mehr Erfahrung
        score += weight_experience * 100
        extra_experience = job_experience - min_years
        score += weight_experience * extra_experience * 10  # Beispiel: 10 Punkte pro zusätzliches Jahr
    else:
        # Teilweise Punkte basierend auf dem Verhältnis
        score += weight_experience * (job_experience / min_years) * 100

    # Qualifikationen bewerten
    req_qualifications = job_req['qualifications']
    qualifications_list = re.split(r'[;, ]', qualifications)  # Trennung durch Semikolon, Komma oder Leerzeichen
    req_qualifications_list = re.split(r'[;, ]', req_qualifications)
    qualifications_count = sum(1 for q in qualifications_list if q.strip().lower() in map(str.lower, req_qualifications_list))
    req_qualifications_count = len(req_qualifications_list)
    if qualifications_count >= req_qualifications_count:
        score += weight_qualifications * 100
    else:
        score += weight_qualifications * (qualifications_count / req_qualifications_count) * 100

    # Sprachkenntnisse bewerten
    language_levels = {
        'muttersprache': 100,
        'bilingual': 100,
        'fließend': 80,
        'fluent': 80,
        'berufliche kenntnisse': 60,
        'professional working proficiency': 60,
        'gute kenntnisse': 50,
        'proficient': 50,
        'konversationssicher': 30,
        'conversational': 30,
        'grundkenntnisse': 10,
        'basic': 10,
        'a1': 10,
        'a2': 20,
        'b1': 30,
        'b2': 40,
        'c1': 50,
        'c2': 100
    }

    # Required languages hardcoded: English C1 and German C1
    req_languages_list = ['englisch(c1)', 'deutsch(c1)']
    language_score = 0

    for lang in req_languages_list:
        match = re.search(rf'(\w+)\s*\((\w+)\)', lang, re.IGNORECASE)
        if match:
            language, level = match.groups()
            if language.lower() in qualifications.lower():
                user_level = re.search(rf'{language}\s*\((\w+)\)', qualifications, re.IGNORECASE)
                if user_level:
                    user_level = user_level.group(1).lower()
                    if user_level == level.lower():
                        language_score += language_levels[level.lower()]
                    else:
                        # Minuspunkte für niedrigere Stufen
                        language_score += language_levels.get(user_level, 0) - (language_levels[level.lower()] - language_levels.get(user_level, 0))
                else:
                    # Minuspunkte, wenn die Sprache nicht auf dem geforderten Niveau ist
                    language_score -= language_levels[level.lower()] / 2
            else:
                # Zusatzpunkte für jede zusätzliche Sprache
                language_score += language_levels.get(level.lower(), 10) / 2

    score += weight_qualifications * language_score / 100  # Normalisierung des Sprachscores

    # Ausbildung bewerten
    education_levels = {
        'ausbildung': 10,
        'abitur': 20,
        'duales studium': 30,
        'bachelor': 40,
        'master': 50,
        'phd': 60,
        'doktor': 60,
        'quereinstieg': 15,
        'meister': 35,
        'apprenticeship': 10,
        'high school diploma': 20,
        'dual study': 30,
        'bachelor\'s degree': 40,
        'master\'s degree': 50,
        'doctorate': 60,
        'career change': 15,
        'master craftsman': 35
    }

    req_education = job_req['education']
    education_score = 0
    for level, points in education_levels.items():
        if level in education.lower():
            education_score = points
            break

    score += weight_education * education_score

    # Ensure the maximum score is 100
    score = min(score, 100)

    return score, job

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session, jsonify, send_from_directory
import os
import pdfplumber
import random
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import sqlite3
import shutil
import pyotp
import qrcode
from io import BytesIO
from flask import Flask, render_template
from werkzeug.security import check_password_hash
from flask import Flask, render_template, request, redirect, url_for, session, jsonify
from models import db, User
from werkzeug.security import generate_password_hash, check_password_hash
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from datetime import datetime, timedelta
from models import  User
from flask import flash
app = Flask(__name__)
from flask import Flask, render_template, request, redirect, url_for, flash, session
from deep_translator import GoogleTranslator
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification


SCOPES = ["https://www.googleapis.com/auth/gmail.readonly", "https://www.googleapis.com/auth/gmail.compose", "https://www.googleapis.com/auth/gmail.send", "https://www.googleapis.com/auth/gmail.modify"]
app.secret_key = 'abc123'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)
migrate = Migrate(app, db)

################### KI-Helper########################################################################
def extract_contacts(text):
    email_pattern = (
        r'\b'
        r'[A-Za-z0-9._%+-]+'
        r'@'
        r'[A-Za-z0-9.-]+'
        r'\.'
        r'[A-Z|a-z]{2,}'
        r'\b'
    )
    phone_pattern = (
        r'(?:(?:\+|00)\d{1,3}[-.\s]?)?' 
        r'\(?(?:\d{2,5})\)?'             
        r'[-.\s]?\d{3,4}'                
        r'[-.\s]?\d{3,4}'
        r'(?:[-.\s]?\d{1,9})?'
    )
    birthday_pattern = (
        r'\b'
        r'\d{2}[./-]\d{2}[./-]\d{4}'
        r'\b'
    )
    
    emails = re.findall(email_pattern, text)
    phones = re.findall(phone_pattern, text)
    birthdays = re.findall(birthday_pattern, text)

    return {
        "E-Mail-Adressen": emails,
        "Telefonnummern": phones,
        "Geburtstag": birthdays
    }

def extract_person_name(text):
    nlp = spacy.load("de_core_news_sm")
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PER" and not any(token.dep_ in ["prep", "pobj"] for token in ent)]
    return persons

def extract_experience_and_knowledge(text):
    MODEL_PATH = r"C:\Users\DE132809\Documents\6. Semester\ZDT\new_modell"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
    tokens = tokenizer(text.split(), return_tensors="pt", is_split_into_words=True, truncation=True, padding='max_length', max_length=512)
    with torch.no_grad():
        outputs = model(**tokens)
        predictions = torch.argmax(outputs.logits, dim=2)
    id2label = model.config.id2label
    predicted_labels = [id2label[label_id] for label_id in predictions[0].numpy()]
    def extract_experience_phrases(tokens, labels):
        phrases = []
        current_phrase = []
        inside_phrase = False
        for token, label in zip(tokens, labels):
            if token in ['[PAD]', '[CLS]', '[SEP]']:
                continue
            if token.startswith("##"):
                if current_phrase:
                    current_phrase[-1] += token[2:]
                continue
            if label == 'B-BERUFSERFAHRUNG':
                if current_phrase:
                    phrases.append(" ".join(current_phrase))
                    current_phrase = []
                current_phrase.append(token)
                inside_phrase = True
            elif label == 'I-BERUFSERFAHRUNG' and inside_phrase:
                current_phrase.append(token)
            else:
                if current_phrase:
                    phrases.append(" ".join(current_phrase))
                    current_phrase = []
                inside_phrase = False
        if current_phrase:
            phrases.append(" ".join(current_phrase))
        return phrases
    def extract_knowledge(tokens, labels):
        knowledge = []
        for token, label in zip(tokens, labels):
            if token in ['[PAD]', '[CLS]', '[SEP]']:
                continue
            if token.startswith("##"):
                if knowledge:
                    knowledge[-1] += token[2:]
                continue
            if label in ['B-KENNTNISSE', 'I-KENNTNISSE']:
                knowledge.append(token)
        return ", ".join(knowledge)
    experience_phrases = extract_experience_phrases(tokens.tokens(), predicted_labels)
    knowledge = extract_knowledge(tokens.tokens(), predicted_labels)
    return experience_phrases, knowledge

def extract_education_level(text, src_lang='de', dest_lang='en'):
    translator = GoogleTranslator(source=src_lang, target=dest_lang)
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    
    def answer_question(question, context):
        inputs = tokenizer(question, context, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
        return answer
    
    ask_bachelor = "Bachelor" in text
    ask_master = "Master" in text
    ask_vocational_training = "abgeschlossene Berufsausbildung" in text
    ask_phd = "Doktor" in text or "PhD" in text
    translated_cv_text = translator.translate(text)
    questions = []
    
    if ask_bachelor:
        questions.append("What is the applier's Bachelor degree?")
    if ask_master:
        questions.append("What is the applier's Master degree?")
    if ask_vocational_training:
        questions.append("What vocational training does Max Mustermann have?")
    if ask_phd:
        questions.append("What is the applier's Doctoral degree?")
    
    answers = {}
    for question in questions:
        answer = answer_question(question, translated_cv_text)
        translator_back = GoogleTranslator(source=dest_lang, target=src_lang)
        translated_answer = translator_back.translate(answer)
        answers[question] = translated_answer
    
    full_answer = ""
    if "What is the applier's Bachelor degree?" in answers:
        full_answer += "Bachelor in " + answers["What is the applier's Bachelor degree?"] + "\n"
    if "What is the applier's Master degree?" in answers:
        full_answer += "Master in " + answers["What is the applier's Master degree?"] + "\n"
    if "What vocational training does Max Mustermann have?" in answers:
        full_answer += "abgeschlossene Berufsausbildung zum " + answers["What vocational training does Max Mustermann have?"] + "\n"
    if "What is the applier's Doctoral degree?" in answers:
        full_answer += "Doktor in " + answers["What is the applier's Doctoral degree?"] + "\n"
    
    return full_answer.strip()

def translate_text(text, src_lang='de', dest_lang='en'):
    translator = GoogleTranslator(source=src_lang, target=dest_lang)
    translation = translator.translate(text)
    return translation

def answer_question_with_t5(question, context):
    tokenizer = T5Tokenizer.from_pretrained('t5-small')
    model = T5ForConditionalGeneration.from_pretrained('t5-small')
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(input_ids)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


def extract_info_from_pdf(file):
    with pdfplumber.open(file) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
    info = {
        "Vorname": "",
        "Nachname": "",
        "Job": "test",
        "E-Mail": "",
        "Geburtstag": "",
        "Berufserfahrung": "",
        "Qualifikationen": "",
        "Ausbildung": ""
    }
    names = extract_person_name(text)
    if len(names) > 0:
        split_name = names[0].split()
        if len(split_name) > 1:
            info["Vorname"] = split_name[0]
            info["Nachname"] = ' '.join(split_name[1:])
        else:
            info["Vorname"] = split_name[0]
    contacts = extract_contacts(text)
    if len(contacts["E-Mail-Adressen"]) > 0:
        info["E-Mail"] = contacts["E-Mail-Adressen"][0]
    if len(contacts["Geburtstag"]) > 0:
        info["Geburtstag"] = contacts["Geburtstag"][0]
    experiences, knowledge = extract_experience_and_knowledge(text)
    print(experiences)
    info["Berufserfahrung"] = ', '.join(experiences)
    info["Qualifikationen"] = knowledge
    
    # Sprachkenntnisse extrahieren
    translated_text = translate_text(text, src_lang='de', dest_lang='en')
    language_question = "Which languages and levels does the speaker speak?"
    language_skills = answer_question_with_t5(language_question, translated_text)
    language_skills = translate_text(language_skills, src_lang='en', dest_lang='de')
    
    if info["Qualifikationen"]:
        info["Qualifikationen"] += ", " + language_skills
    else:
        info["Qualifikationen"] = language_skills
    
    education = extract_education_level(text)
    info["Ausbildung"] = education
    return info

######################################################################################################################################################

def get_2fa_qr_code(user):
    """
    Erzeugt einen QR-Code für die 2FA-Provisioning-URL des Benutzers.
    """
    # Erstelle ein TOTP-Objekt mit dem im Benutzer gespeicherten Geheimnis
    totp = pyotp.TOTP(user.two_factor_secret)
    # Erzeuge die Provisioning-URL (passe den Issuer-Namen an deinen App-Namen an)
    provisioning_url = totp.provisioning_uri(name=user.email, issuer_name="DeineApp")
    
    # Erzeuge einen QR-Code aus der URL
    qr = qrcode.QRCode(border=1)
    qr.add_data(provisioning_url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    
    # Konvertiere das Bild in einen base64-kodierten String
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return img_str, provisioning_url

@app.route('/')
def index():
    try:
        return render_template('index.html')
    except Exception as e:
        return str(e)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        token = request.form.get('token')  # Vom Benutzer eingegebener TOTP-Code

        # Benutzer anhand der E-Mail abrufen
        user = User.query.filter_by(email=email).first()

        if user and check_password_hash(user.password, password):
            # Falls 2FA aktiviert ist (Geheimnis gesetzt), TOTP-Code prüfen
            if user.two_factor_secret:
                totp = pyotp.TOTP(user.two_factor_secret)
                if not token or not totp.verify(token):
                    flash("Ungültiger 2FA-Code", "error")
                    return redirect(url_for('login'))
            # Benutzer ist authentifiziert: Speichere die User-ID in der Session
            session['user_id'] = user.id
            flash("Login erfolgreich", "success")
            return redirect(url_for('stellenanzeigen'))
        else:
            flash("Ungültige Anmeldedaten", "error")
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        hashed_password = generate_password_hash(password)
        # Erstelle den Benutzer und weise gleich ein 2FA-Geheimnis zu
        user = User(email=email, password=hashed_password)
        user.two_factor_secret = pyotp.random_base32()  # Generiert ein zufälliges Geheimnis
        db.session.add(user)
        db.session.commit()
        
        # Speichere die Benutzer-ID in der Session
        session['user_id'] = user.id
        
        flash("Registrierung erfolgreich. Bitte richten Sie jetzt die Zwei-Faktor-Authentifizierung ein.", "info")
        return redirect(url_for('setup_2fa'))
    return render_template('register.html')

def get_2fa_qr_code(user):
    """
    Erzeugt einen QR-Code für die TOTP-Provisioning-URL des Benutzers.
    """
    totp = pyotp.TOTP(user.two_factor_secret)
    # Passe den issuer_name an (z. B. den Namen deiner Anwendung)
    provisioning_url = totp.provisioning_uri(name=user.email, issuer_name="DeineApp")
    
    # Erzeuge den QR-Code
    qr = qrcode.QRCode(border=1)
    qr.add_data(provisioning_url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    
    # Konvertiere den QR-Code in einen Base64-kodierten String
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return img_str, provisioning_url

@app.route('/setup_2fa', methods=['GET', 'POST'])
def setup_2fa():
    # Hole die Benutzer-ID aus der Session
    user_id = session.get('user_id')
    if not user_id:
        flash("Bitte loggen Sie sich ein.", "error")
        return redirect(url_for('login'))
    
    user = User.query.get(user_id)
    if not user:
        flash("Benutzer nicht gefunden.", "error")
        return redirect(url_for('login'))
    
    if request.method == 'POST':
        # Der Benutzer hat den TOTP-Code eingegeben, um 2FA zu verifizieren.
        token = request.form.get('token')
        totp = pyotp.TOTP(user.two_factor_secret)
        if totp.verify(token):
            flash("2FA erfolgreich eingerichtet!", "success")
            return redirect(url_for('stellenanzeigen'))
        else:
            flash("Ungültiger 2FA-Code. Bitte versuchen Sie es erneut.", "error")
            # Bleibe auf der gleichen Seite, um einen neuen Versuch zu ermöglichen.
    
    # Generiere den QR-Code zum Anzeigen
    qr_code, provisioning_url = get_2fa_qr_code(user)
    return render_template('setup_2fa.html', qr_code=qr_code, secret=user.two_factor_secret)



@app.route('/logout')
def logout():
    session.pop('logged_in', None)  # Entferne den Login-Status aus der Session
    return redirect(url_for('login'))

@app.route('/bewerbung')
def bewerbung():
    return render_template('bewerbung.html')

@app.route('/dashboard', methods=['GET', 'POST'])
def dashboard():
    """if not session.get('logged_in'):  # Überprüfe den Login-Status
        return redirect(url_for('login'))"""

    # Verbindung zur Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten aus der Datenbank abfragen
    cursor.execute('SELECT id, vorname || " " || nachname AS name, score, job AS position FROM employees')
    scores = [{'id': row[0], 'name': row[1], 'score': row[2], 'position': row[3]} for row in cursor.fetchall()]

    # Einzigartige Job-Positionen aus der Datenbank abfragen
    cursor.execute('SELECT DISTINCT job FROM employees')
    jobs = [row[0] for row in cursor.fetchall()]

    # Verbindung schließen
    conn.close()

    # 'Alle' zur Liste der Positionen hinzufügen
    positions = ['Alle'] + jobs
    
    selected_position = request.form.get('position', 'Alle')
    if selected_position == 'Alle':
        filtered_scores = scores
    else:
        filtered_scores = [score for score in scores if score['position'] == selected_position]
    
    return render_template('dashboard.html', scores=filtered_scores, positions=positions, selected_position=selected_position)

@app.route('/update_scores', methods=['POST'])
def update_scores():
    selected_position = request.json.get('selected_position', 'Alle')

    # Verbindung zur Bewerber-Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten aus der Datenbank abfragen
    cursor.execute('SELECT id, vorname || " " || nachname AS name, score, job AS position FROM employees')
    scores = [{'id': row[0], 'name': row[1], 'score': row[2], 'position': row[3]} for row in cursor.fetchall()]

    # PDF-Dateien aus dem Ordner lesen und Informationen extrahieren
    pdf_folder = 'attachments'
    processed_folder = 'Bearbeitete_Dokumente'
    os.makedirs(processed_folder, exist_ok=True)

    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            info = extract_info_from_pdf(file_path)

            # Beispielhafte Einfügung in die Datenbank (hier nur für Demonstrationszwecke)
            score, job = calculate_score(info["Job"], info["Berufserfahrung"], info["Qualifikationen"], info["Ausbildung"])
            cursor.execute('INSERT INTO employees (job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
                           (info["Job"], info["Vorname"], info["Nachname"], info["E-Mail"], info["Geburtstag"], info["Berufserfahrung"], info["Qualifikationen"], info["Ausbildung"], score))

            # PDF-Datei in den Ordner Bearbeitete_Dokumente verschieben
            shutil.move(file_path, os.path.join(processed_folder, filename))

    # Verbindung schließen
    conn.commit()
    conn.close()

    if selected_position != 'Alle':
        scores = [score for score in scores if score['position'] == selected_position]

    # Erfolgsmeldung hinzufügen
    response = {
        'message': 'Aktualisierung erfolgreich!',
        'scores': scores
    }

    return jsonify(response)

@app.route('/delete/<int:id>', methods=['POST'])
def delete(id):
    # Verbindung zur Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Eintrag löschen
    cursor.execute('DELETE FROM employees WHERE id = ?', (id,))
    conn.commit()

    # Verbindung schließen
    conn.close()

    return redirect(url_for('dashboard'))


@app.route('/api/extract_resume', methods=['POST'])
def extract_resume():
    file = request.files['resume']
    info = extract_info_from_pdf(file)

    result = {
        'firstname': info["Vorname"],
        'lastname': info["Nachname"],
        'birthdate': info["Geburtstag"],
        'job': info["Job"],
        'email': info["E-Mail"],
        'experience': info["Berufserfahrung"],
        'qualifications': info["Qualifikationen"],
        'education': info["Ausbildung"]
    }

    return jsonify(result)

@app.route('/api/fetch_mails', methods=['GET'])
def fetch_mails():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("gmail", "v1", credentials=creds)
        messages = get_unread_emails_with_subject(service, "me", "Bewerbung")
        store_dir = "attachments"
        if not os.path.exists(store_dir):
            os.makedirs(store_dir)
        for msg in messages:
            get_attachments(service, "me", msg['id'], store_dir)
            mark_as_read(service, "me", msg['id'])
        return jsonify({"status": "success", "message": "Mails wurden abgerufen und gespeichert."})
    except HttpError as error:
        return jsonify({"status": "error", "message": str(error)})

@app.route('/submit_application', methods=['POST'])
def submit_application():
    job = request.form.get('job')
    firstname = request.form.get('firstname')
    lastname = request.form.get('lastname')
    email = request.form.get('email')
    birthdate = request.form.get('birthdate')
    experience = request.form.get('experience')
    qualifications = request.form.get('qualifications')
    education = request.form.get('education')
    file = request.files['resume']

    # Score berechnen
    score, job = calculate_score(job, experience, qualifications, education)

    # Verbindung zur Bewerber-Datenbank herstellen
    conn = conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten in die Datenbank einfügen
    cursor.execute('''
        INSERT INTO employees (job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (job, firstname, lastname, email, birthdate, experience, qualifications, education, score))
    conn.commit()
    conn.close()

    processed_folder = 'Bearbeitete_Dokumente'
    os.makedirs(processed_folder, exist_ok=True)
    
    filename = file.filename
    file_path = os.path.join(processed_folder, filename)
    base, extension = os.path.splitext(filename)
    counter = 1
    while os.path.exists(file_path):
        new_filename = f"{base}_{counter}{extension}"
        file_path = os.path.join(processed_folder, new_filename)
        counter += 1

    file.save(file_path)

    return jsonify({'status': 'success', 'message': 'Daten erfolgreich eingetragen'})

@app.route('/get_email/<int:id>', methods=['GET'])
def get_email(id):
    # Verbindung zur Bewerber-Datenbank herstellen
    conn1 = sqlite3.connect('bewerber.db')
    cursor1 = conn1.cursor()

    # Informationen basierend auf der ID abfragen
    cursor1.execute('''
        SELECT job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score 
        FROM employees 
        WHERE id = ?
    ''', (id,))
    data1 = cursor1.fetchone()

    # Verbindung zur Bewerber-Datenbank schließen
    conn1.close()

    if data1:
        # Verbindung zur Job-Anforderungen-Datenbank herstellen
        conn2 = sqlite3.connect('job_requirements.db')
        conn2.row_factory = sqlite3.Row
        cursor2 = conn2.cursor()

        # Job-Anforderungen basierend auf dem Jobnamen abfragen
        cursor2.execute('''
            SELECT jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education 
            FROM job_requirements 
            WHERE jobname = ?
        ''', (data1[0],))
        data2 = cursor2.fetchone()

        # Verbindung zur Job-Anforderungen-Datenbank schließen
        conn2.close()

        if data2:
            return jsonify({
                'job': data1[0],
                'vorname': data1[1],
                'nachname': data1[2],
                'email': data1[3],
                'geburtstag': data1[4],
                'berufserfahrung': data1[5],
                'qualifikation': data1[6],
                'ausbildung': data1[7],
                'score': data1[8],
                'jobname': data2['jobname'],
                'experience': data2['experience'],
                'qualifications': data2['qualifications'],
                'education': data2['education'],
                'location': data2['location'],
                'weight_experience': data2['weight_experience'],
                'weight_qualifications': data2['weight_qualifications'],
                'weight_education': data2['weight_education']
            })
        else:
            return jsonify({'error': 'Job-Anforderungen nicht gefunden'}), 404
    else:
        return jsonify({'error': 'Daten nicht gefunden'}), 404



    
@app.route('/manage_jobs')
def manage_jobs():
    """if not session.get('logged_in'):  # Überprüfe den Login-Status
        return redirect(url_for('login'))"""
    conn = get_db_connection()
    jobs = conn.execute('SELECT * FROM job_requirements').fetchall()
    conn.close()
    return render_template('jobverwaltung.html', jobs=jobs)

@app.route('/create_job', methods=['POST'])
def create_job():
    jobname = request.form['jobname']
    experience = request.form['experience']
    qualifications = request.form['qualifications']
    education = request.form['education']
    location = request.form['location']
    weight_experience = float(request.form['weight_experience'])
    weight_qualifications = float(request.form['weight_qualifications'])
    weight_education = float(request.form['weight_education'])

    conn = get_db_connection()
    conn.execute('''
        INSERT INTO job_requirements (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education))
    conn.commit()
    job_id = conn.execute('SELECT last_insert_rowid()').fetchone()[0]
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich erstellt!', 'job': {'id': job_id, 'jobname': jobname}})

@app.route('/job_details/<int:job_id>')
def job_details(job_id):
    conn = get_db_connection()
    job = conn.execute('SELECT * FROM job_requirements WHERE id = ?', (job_id,)).fetchone()
    conn.close()
    if job is None:
        return jsonify({'status': 'error', 'message': 'Job nicht gefunden!'})
    return jsonify({
        'jobname': job['jobname'],
        'experience': job['experience'],
        'qualifications': job['qualifications'],
        'education': job['education'],
        'location': job['location'],
        'weight_experience': job['weight_experience'],
        'weight_qualifications': job['weight_qualifications'],
        'weight_education': job['weight_education']
    })

@app.route('/update_job/<int:job_id>', methods=['POST'])
def update_job(job_id):
    jobname = request.form['jobname']
    experience = request.form['experience']
    qualifications = request.form['qualifications']
    education = request.form['education']
    location = request.form['location']
    weight_experience = float(request.form['weight_experience'])
    weight_qualifications = float(request.form['weight_qualifications'])
    weight_education = float(request.form['weight_education'])

    conn = get_db_connection()
    conn.execute('''
        UPDATE job_requirements
        SET jobname = ?, experience = ?, qualifications = ?, education = ?, location = ?, weight_experience = ?, weight_qualifications = ?, weight_education = ?
        WHERE id = ?
    ''', (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education, job_id))
    conn.commit()
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich aktualisiert!'})

@app.route('/delete_job/<int:job_id>', methods=['DELETE'])
def delete_job(job_id):
    conn = get_db_connection()
    conn.execute('DELETE FROM job_requirements WHERE id = ?', (job_id,))
    conn.commit()
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich gelöscht!'})

@app.route('/stellenanzeigen')
def stellenanzeigen():
    conn = get_db_connection()
    jobs = conn.execute('SELECT * FROM job_requirements').fetchall()
    conn.close()
    return render_template('stellenanzeigen.html', jobs=jobs)

@app.route('/job_details_popup/<int:job_id>')
def job_details_popup(job_id):
    conn = get_db_connection()
    job = conn.execute('SELECT * FROM job_requirements WHERE id = ?', (job_id,)).fetchone()
    conn.close()
    if job is None:
        return jsonify({'status': 'error', 'message': 'Job nicht gefunden!'})
    return jsonify({
        'jobname': job['jobname'],
        'experience': job['experience'],
        'qualifications': job['qualifications'],
        'education': job['education'],
        'location': job['location']
    })

@app.route('/download_template')
def download_template():
    return send_from_directory(directory='.', path="Vorlage_Bewerbung.dotx", as_attachment=True)

@app.route('/api/jobs', methods=['GET'])
def get_jobs():
    conn = sqlite3.connect('job_requirements.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    cursor.execute('SELECT jobname FROM job_requirements')
    jobs = cursor.fetchall()
    job_list = [{"jobname": row["jobname"]} for row in jobs]

    conn.close()
    return jsonify(job_list)
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Feb/2025 13:05:27] "GET /bewerbung HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2025 13:05:27] "GET /static/Logo.jpg HTTP/1.1" 304 -
127.0.0.1 - - [18/Feb/2025 13:05:27] "GET /api/jobs HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2025 13:05:27] "GET /static/favicon.ico HTTP/1.1" 304 -


['abgeschlossene berufsausbildung zum fahrradmechtroniker', '2 jahre als schweißer', '3 jahre als busfahrer']


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
127.0.0.1 - - [18/Feb/2025 13:05:38] "POST /api/extract_resume HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2025 13:06:07] "POST /api/extract_resume HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\DE132809\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1498, in __call__
    re